In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

### 读取预处理好的数据：

In [2]:
train_data = pd.read_csv('data/train_data_v1.csv', sep=' ')
test_data = pd.read_csv('data/test_data_v1.csv', sep=' ')

### 训练数据浏览：

In [3]:
train_data.head()

,SaleID,bodyType,brand,creatDate,fuelType,gearbox,kilometer,model,notRepairedDamage,power,...,v_9,name_count,regDates,creatDates,regDate_year,regDate_month,regDate_day,creatDate_year,creatDate_month,creatDate_day
0,0,1.0,6,20160404,0.0,0.0,12.5,30.0,0.0,60,...,0.097462,108,2004-04-02,2016-04-04,2004,4,2,2016,4,4
1,1,2.0,1,20160309,0.0,0.0,15.0,40.0,0.0,0,...,0.020582,29,2003-03-01,2016-03-09,2003,3,1,2016,3,9
2,2,1.0,15,20160402,0.0,0.0,12.5,115.0,0.0,163,...,0.027075,3,2004-04-03,2016-04-02,2004,4,3,2016,4,2
3,3,0.0,10,20160312,0.0,1.0,15.0,109.0,0.0,193,...,0.000000,2,1996-09-08,2016-03-12,1996,9,8,2016,3,12
4,4,1.0,5,20160313,0.0,0.0,5.0,110.0,0.0,68,...,0.121534,1,2012-01-03,2016-03-13,2012,1,3,2016,3,13


### 测试数据浏览：

In [4]:
test_data.head()

,SaleID,bodyType,brand,creatDate,fuelType,gearbox,kilometer,model,notRepairedDamage,power,...,v_9,name_count,regDates,creatDates,regDate_year,regDate_month,regDate_day,creatDate_year,creatDate_month,creatDate_day
0,200000,1.0,0,20160308,0.0,0.0,15.0,67.0,0.0,101,...,0.094522,1,2000-05-01,2016-03-08,2000,5,1,2016,3,8
1,200001,2.0,6,20160310,0.0,0.0,6.0,19.0,0.0,73,...,0.035385,1,1995-02-11,2016-03-10,1995,2,11,2016,3,10
2,200002,4.0,5,20160309,0.0,0.0,5.0,5.0,0.0,120,...,0.073586,1,2009-06-06,2016-03-09,2009,6,6,2016,3,9
3,200003,1.0,9,20160314,0.0,0.0,15.0,22.0,0.0,58,...,0.096614,71,2002-06-01,2016-03-14,2002,6,1,2016,3,14
4,200004,0.0,6,20160306,0.0,0.0,15.0,46.0,0.0,116,...,0.068852,1,2003-03-01,2016-03-06,2003,3,1,2016,3,6


### 训练数据的统计信息：

In [5]:
train_data.describe()

,SaleID,bodyType,brand,creatDate,fuelType,gearbox,kilometer,model,notRepairedDamage,power,...,v_7,v_8,v_9,name_count,regDate_year,regDate_month,regDate_day,creatDate_year,creatDate_month,creatDate_day
count,149999.000000,149999.000000,149999.000000,1.499990e+05,149999.000000,149999.000000,149999.000000,149999.000000,149999.000000,149999.000000,...,149999.000000,149999.000000,149999.000000,149999.000000,149999.000000,149999.000000,149999.000000,149999.000000,149999.000000,149999.000000
mean,74999.493837,1.738525,8.052727,2.016033e+07,0.354096,0.215975,12.597144,47.128581,0.095434,116.861752,...,0.124693,0.058144,0.061995,16.621251,2003.357196,5.998393,6.502863,2015.999880,3.161581,15.833826
std,43301.558800,1.760784,7.864982,1.067332e+02,0.539748,0.411498,3.919584,49.536165,0.293814,70.074840,...,0.201410,0.029185,0.035692,48.697958,5.362246,3.521590,3.450316,0.010954,0.380710,9.132285
min,0.000000,0.000000,0.000000,2.015062e+07,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,1991.000000,1.000000,1.000000,2015.000000,1.000000,1.000000
25%,37499.500000,0.000000,1.000000,2.016031e+07,0.000000,0.000000,12.500000,10.000000,0.000000,75.000000,...,0.062474,0.035334,0.033930,1.000000,1999.000000,3.000000,4.000000,2016.000000,3.000000,8.000000
50%,74999.000000,1.000000,6.000000,2.016032e+07,0.000000,0.000000,15.000000,30.000000,0.000000,110.000000,...,0.095867,0.057014,0.058483,1.000000,2003.000000,6.000000,7.000000,2016.000000,3.000000,16.000000
75%,112499.500000,3.000000,13.000000,2.016033e+07,1.000000,0.000000,15.000000,66.000000,0.000000,150.000000,...,0.125243,0.079382,0.087489,7.000000,2007.000000,9.000000,9.000000,2016.000000,3.000000,24.000000
max,149999.000000,7.000000,39.000000,2.016041e+07,6.000000,1.000000,15.000000,247.000000,1.000000,600.000000,...,1.404936,0.160791,0.222787,376.000000,2015.000000,12.000000,12.000000,2016.000000,12.000000,31.000000


### 测试数据的统计信息：

In [6]:
test_data.describe()

,SaleID,bodyType,brand,creatDate,fuelType,gearbox,kilometer,model,notRepairedDamage,power,...,v_7,v_8,v_9,name_count,regDate_year,regDate_month,regDate_day,creatDate_year,creatDate_month,creatDate_day
count,50000.000000,50000.000000,50000.000000,5.000000e+04,50000.00000,50000.000000,50000.000000,50000.00000,50000.000000,50000.00000,...,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.00000,50000.000000
mean,224999.500000,1.739780,8.087140,2.016033e+07,0.35448,0.218020,12.598260,47.64948,0.094140,116.51788,...,0.124693,0.058198,0.062113,16.751520,2003.341440,5.977240,6.507780,2015.999920,3.16138,15.864040
std,14433.901067,1.765018,7.899648,1.113395e+02,0.54025,0.412905,3.912519,49.90741,0.292026,70.48107,...,0.201440,0.029171,0.035723,48.826943,5.348883,3.520899,3.437385,0.010954,0.37802,9.145523
min,200000.000000,0.000000,0.000000,2.014031e+07,0.00000,0.000000,0.500000,0.00000,0.000000,0.00000,...,0.000000,0.000000,0.000000,1.000000,1991.000000,1.000000,1.000000,2014.000000,1.00000,1.000000
25%,212499.750000,0.000000,1.000000,2.016031e+07,0.00000,0.000000,12.500000,11.00000,0.000000,75.00000,...,0.062519,0.035413,0.033880,1.000000,1999.000000,3.000000,4.000000,2016.000000,3.00000,8.000000
50%,224999.500000,1.000000,6.000000,2.016032e+07,0.00000,0.000000,15.000000,30.00000,0.000000,110.00000,...,0.095880,0.056804,0.058749,1.000000,2003.000000,6.000000,7.000000,2016.000000,3.00000,16.000000
75%,237499.250000,3.000000,13.000000,2.016033e+07,1.00000,0.000000,15.000000,66.00000,0.000000,150.00000,...,0.125470,0.079387,0.087624,7.000000,2007.000000,9.000000,9.000000,2016.000000,3.00000,24.000000
max,249999.000000,7.000000,39.000000,2.016041e+07,6.00000,1.000000,15.000000,246.00000,1.000000,600.00000,...,1.411559,0.157458,0.211304,376.000000,2015.000000,12.000000,12.000000,2016.000000,12.00000,31.000000


### 提取所需特征列：

In [7]:
feature_cols = [x for x in train_data.columns if x not in ['SaleID','name','regDates','creatDates','price','model','brand','regionCode','seller', 'Type','creatDate']]

### 构建训练和测试数据集：

In [8]:
train_X = train_data[feature_cols]
test_X = test_data[feature_cols]
train_Y = train_data['price']

In [9]:
print('X train shape:',train_X.shape)
print('X test shape:',test_X.shape)
print('Y train shape:',train_Y.shape)

X train shape: (149999, 29)
X test shape: (50000, 29)
Y train shape: (149999,)


### 切分数据集（Train,Val）进行模型训练，评价和预测：

In [10]:
x_train, x_val, y_train, y_val = train_test_split(train_X, train_Y, test_size=0.3)

### 使用线性回归模型进行训练，使用MAE标准进行评价：

In [11]:
train_model = LinearRegression(normalize=True)
train_model.fit(x_train, y_train)
predictions = train_model.predict(x_val)
mae = mean_absolute_error(predictions, y_val)
print('MAE = ', mae)

MAE =  0.19382181965562809


### 模型预测：

In [12]:
pred_model = LinearRegression(normalize=True)
pred_model.fit(train_X, train_Y)
price = pred_model.predict(test_X)

### 将预测值生成指定格式的csv文件：

In [13]:
submit = pd.DataFrame()
submit['SaleID'] = test_data.SaleID
submit['price'] = price
submit.to_csv('output/submit.csv',index=False)

In [14]:
submit.head(10)

,SaleID,price
0,200000,7.060200
1,200001,7.427987
2,200002,9.122695
3,200003,7.103163
4,200004,7.637307
5,200005,7.007453
6,200006,6.129283
7,200007,8.105281
8,200008,9.466557
9,200009,6.436645
